In [1]:
from numba import cuda
import numpy as np

In [2]:
device = cuda.get_current_device()
print(device.MAX_THREADS_PER_BLOCK)

1024


In [3]:
def return_blocks_and_threads(tpb, ip_size):
    blocks = (ip_size + tpb - 1) // tpb
    return blocks

In [4]:
@cuda.jit
def multiply_by_scalar(arr, scalar, result):
    idx = cuda.grid(1)

    if idx < arr.size:
        result[idx] = arr[idx] * scalar


def  test_multiply_scalar():

    arr = np.random.random(10000000)
    scalar = 8.8
    result = np.zeros_like(arr)

    tpb = 1024
    d_result = cuda.to_device(result)
    multiply_by_scalar[return_blocks_and_threads(tpb, arr.size), tpb](
        cuda.to_device(arr),scalar, d_result)

    print(d_result.copy_to_host())
    

In [5]:
test_multiply_scalar()

[2.2401102  3.34628874 3.92188087 ... 0.7065961  3.68134633 6.07182281]


In [6]:
## Compute euclidean distance between 2 arrays of size N*2
## sqrt((x2-x1)**2 + (y2-y1)**2)

import math

@cuda.jit
def euclidean_distance(ip1, ip2, res):
    idx = cuda.grid(1)

    if idx < res.size:
        dist1 = (ip2[idx,0] - ip1[idx,0]) ** 2
        dist2 = (ip2[idx,1] - ip1[idx,1]) ** 2
        res[idx] = math.sqrt(dist1 + dist2)


def test_distances():
    points1 = np.array([[0, 0], [1, 1], [2, 2], [3, 3]], dtype=np.float32)
    points2 = np.array([[3, 4], [4, 5], [5, 6], [6, 7]], dtype=np.float32)
    res = np.zeros(points1.shape[0])
    d_res = cuda.to_device(res)

    tpb = 1024
    euclidean_distance[return_blocks_and_threads(tpb, points1.shape[0]), tpb](
                        cuda.to_device(points1), cuda.to_device(points2), d_res)

    print(d_res.copy_to_host())
    

    
    

In [ ]:
test_distances() ##Answer is correct 

In [7]:
##Matrix transpose - ip[i,j] will be op[j,i] - Memory access will be intersting 
##as one will read as row major and write to columns i.e. sequential to strided


@cuda.jit
def matrix_transpose_kernel(ip, op):
    row, col = cuda.grid(2)

    if row < ip.shape[0] and col < ip.shape[1]:
        op[col,row] = ip[row,col]


def test_transpose():
    # input_matrix = np.array([
    #     [1,2,3,4],
    #     [5,6,7,8],
    #     [9,10,11,12]
    # ], dtype=np.float32)

    input_matrix = np.random.random((1000,1000))

    output_matrix = np.zeros([input_matrix.shape[1], input_matrix.shape[0]], 
                             dtype=np.float32, order='F') ## Note that this is a column major matrix. Flip it back to row major if using it 
    d_op = cuda.to_device(output_matrix)
    tpb = (32,32)
    blocks_x = (input_matrix.shape[0] + tpb[0] - 1) // tpb[0]
    blocks_y = (input_matrix.shape[1] + tpb[1] - 1) // tpb[1]

    matrix_transpose_kernel[(blocks_x, blocks_y), tpb](cuda.to_device(input_matrix), d_op)

    print(f"Output_matrix {d_op.copy_to_host()}")
    

    



In [8]:
test_transpose()

Output_matrix [[0.56972677 0.43268988 0.55871207 ... 0.40783274 0.9658697  0.17782584]
 [0.11073507 0.05829967 0.24942404 ... 0.5319469  0.8377895  0.9623157 ]
 [0.46692026 0.79620385 0.6737527  ... 0.05810857 0.65877056 0.8411654 ]
 ...
 [0.63199824 0.20264396 0.63260984 ... 0.04771356 0.21630879 0.3447719 ]
 [0.81856406 0.44765735 0.02848713 ... 0.9584645  0.5521447  0.55134827]
 [0.10387079 0.7215964  0.9121541  ... 0.6010241  0.36651507 0.08912477]]


In [9]:
### Shared memory version i.e copy the ip matrix to shared memory and the transpose from shared
#to global
from numba  import float32
TILE_DIM = 32

@cuda.jit
def transpose_in_shared_memory(ip, op):
    tile = cuda.shared.array((TILE_DIM, TILE_DIM), dtype=float32)

    ##This is the Global position i.e. position in global memory
    x = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.y

    ##Copy from global mem to shared mem
    if x < ip.shape[0] and y < ip.shape[1]:
        tile[cuda.threadIdx.x, cuda.threadIdx.y] = ip[x,y]

    cuda.syncthreads()

    ##Swap at block level
    x_trans = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.x
    y_trans = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.y

    if x_trans < op.shape[1] and y_trans < op.shape[0]:
        op[y_trans, x_trans] = tile[cuda.threadIdx.x, cuda.threadIdx.y]


In [10]:
def shared_test_transpose():
    input_matrix = np.random.random((1000,1000))
    output_matrix = np.zeros([input_matrix.shape[1], input_matrix.shape[0]], 
                             dtype=np.float32) 
    d_op = cuda.to_device(output_matrix)
    tpb = (TILE_DIM, TILE_DIM) 
    blocks_x = (input_matrix.shape[0] + tpb[0] - 1) // tpb[0]
    blocks_y = (input_matrix.shape[1] + tpb[1] - 1) // tpb[1]

    transpose_in_shared_memory[(blocks_x, blocks_y), tpb](cuda.to_device(input_matrix), d_op)

    print(f"Output_matrix {d_op.copy_to_host()}")

In [11]:
shared_test_transpose()

Output_matrix [[0.9274719  0.20494366 0.8594856  ... 0.00488816 0.5967963  0.9826212 ]
 [0.285386   0.5006605  0.9118131  ... 0.98209184 0.47903955 0.3684796 ]
 [0.57640207 0.6210588  0.41887552 ... 0.4194919  0.13198346 0.7131573 ]
 ...
 [0.21589097 0.8516272  0.6700764  ... 0.9669673  0.68578225 0.55980283]
 [0.13032268 0.25545824 0.5993233  ... 0.6867984  0.7210074  0.835155  ]
 [0.9522069  0.6852657  0.8695391  ... 0.04512196 0.23766214 0.64739096]]


In [12]:
## Avoid memory bank conflicts - The config could here be probably the worst case scenario
# mainly because my tile dim is 32,32 and memory bank size is also  32. And given that
# I am doing row major access chances are I would have memory bank conflict - Need to run
#nsight compute to see this. But for now let me just pad the cool tile by 1 if you are using
#shared memory

T_DIM = 32
T_DIM_PAD = 33


@cuda.jit
def shared_mem_transpose_pad(ip, op):
    tile = cuda.shared.array((T_DIM, T_DIM_PAD), dtype=float32) #You would just pad the tile

    ##Locate the global co-ordiantes
    x = cuda.blockIdx.x * T_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.y * T_DIM + cuda.threadIdx.y

    if x < ip.shape[0] and y < ip.shape[1]:
        tile[cuda.threadIdx.x, cuda.threadIdx.y] =  ip[x,y]

    cuda.syncthreads()

    ##Get the block co-ordiantes - because you do block swaps
    x_trans = cuda.blockIdx.y * T_DIM + cuda.threadIdx.x
    y_trans = cuda.blockIdx.x * T_DIM + cuda.threadIdx.y

    if x_trans < op.shape[1] and y_trans < op.shape[0]:
        op[y_trans, x_trans] = tile[cuda.threadIdx.x, cuda.threadIdx.y]



In [13]:
def shared_test_transpose_pad():
    # input_matrix = np.array([
    #     [1,2,3,4],
    #     [5,6,7,8],
    #     [9,10,11,12]
    # ], dtype=np.float32)

    input_matrix = np.random.random((1000,1000))

    output_matrix = np.zeros([input_matrix.shape[1], input_matrix.shape[0]], 
                             dtype=np.float32) 
    d_op = cuda.to_device(output_matrix)
    tpb = (T_DIM, T_DIM) 
    blocks_x = (input_matrix.shape[0] + tpb[0] - 1) // tpb[0]
    blocks_y = (input_matrix.shape[1] + tpb[1] - 1) // tpb[1]

    shared_mem_transpose_pad[(blocks_x, blocks_y), tpb](cuda.to_device(input_matrix), d_op)

    print(f"Output_matrix {d_op.copy_to_host()}")

In [14]:
shared_test_transpose_pad()

Output_matrix [[0.5930592  0.71818256 0.25634786 ... 0.41163415 0.0057651  0.3681336 ]
 [0.19793984 0.43368843 0.8564677  ... 0.9042381  0.13848855 0.9266373 ]
 [0.7113977  0.87533164 0.9835797  ... 0.28380236 0.01566118 0.2015533 ]
 ...
 [0.62743753 0.27151245 0.32401225 ... 0.0584616  0.8580972  0.11938863]
 [0.44579387 0.49021393 0.14080866 ... 0.11996321 0.02662177 0.4002938 ]
 [0.6483071  0.43076864 0.52313876 ... 0.429559   0.281192   0.92398036]]


In [ ]:
#Viewing PTX code

In [15]:
for k,v in multiply_by_scalar.inspect_asm().items():
    print(f"For signature {k}")
    print(f"\n The assembly is \n {v}")
    print("END")

For signature (Array(float64, 1, 'C', False, aligned=True), float64, Array(float64, 1, 'C', False, aligned=True))

 The assembly is 
 //
// Generated by NVIDIA NVVM Compiler
//
// Compiler Build ID: CL-34097967
// Cuda compilation tools, release 12.4, V12.4.131
// Based on NVVM 7.0.1
//

.version 8.4
.target sm_61
.address_size 64

	// .globl	_ZN6cudapy8__main__18multiply_by_scalarB2v1B96cw51cXTLSUwv1sCUt9Uw01Ew1NRRQPKiLTj0gIGIFp_2b2oLQFEYYkHSQB1OQAk0Bynm21OizQ1K0UoIGvDpQE8oxrNQE_3dE5ArrayIdLi1E1C7mutable7alignedEd5ArrayIdLi1E1C7mutable7alignedE
.visible .global .align 4 .u32 _ZN6cudapy8__main__18multiply_by_scalarB2v1B96cw51cXTLSUwv1sCUt9Uw01Ew1NRRQPKiLTj0gIGIFp_2b2oLQFEYYkHSQB1OQAk0Bynm21OizQ1K0UoIGvDpQE8oxrNQE_3dE5ArrayIdLi1E1C7mutable7alignedEd5ArrayIdLi1E1C7mutable7alignedE__errcode__;
.visible .global .align 4 .u32 _ZN6cudapy8__main__18multiply_by_scalarB2v1B96cw51cXTLSUwv1sCUt9Uw01Ew1NRRQPKiLTj0gIGIFp_2b2oLQFEYYkHSQB1OQAk0Bynm21OizQ1K0UoIGvDpQE8oxrNQE_3dE5ArrayIdLi1E1C7mutable7a

In [16]:
#This is to check if fma operations are used instead of typical multiply and then add

@cuda.jit
def multiply_by_scalar_and_add(arr, scalar, result):
    idx = cuda.grid(1)

    if idx < arr.size:
        result[idx] = (arr[idx] * scalar) + scalar

In [17]:
def test_multiply_scalar_and_add():
    arr = np.random.random(10000000)
    scalar = 8.8
    result = np.zeros_like(arr)

    tpb = 1024
    d_result = cuda.to_device(result)
    multiply_by_scalar_and_add[return_blocks_and_threads(tpb, arr.size), tpb](
        cuda.to_device(arr),scalar, d_result)

    print(d_result.copy_to_host())

In [18]:
test_multiply_scalar_and_add()

[14.19856061 11.04312628 11.38904771 ...  9.80525965 17.13052089
 12.55831722]


In [19]:
for k, v in multiply_by_scalar_and_add.inspect_asm().items():
    print(v)

//
// Generated by NVIDIA NVVM Compiler
//
// Compiler Build ID: CL-34097967
// Cuda compilation tools, release 12.4, V12.4.131
// Based on NVVM 7.0.1
//

.version 8.4
.target sm_61
.address_size 64

	// .globl	_ZN6cudapy8__main__26multiply_by_scalar_and_addB2v5B96cw51cXTLSUwv1sCUt9Uw01Ew1NRRQPKiLTj0gIGIFp_2b2oLQFEYYkHSQB1OQAk0Bynm21OizQ1K0UoIGvDpQE8oxrNQE_3dE5ArrayIdLi1E1C7mutable7alignedEd5ArrayIdLi1E1C7mutable7alignedE
.visible .global .align 4 .u32 _ZN6cudapy8__main__26multiply_by_scalar_and_addB2v5B96cw51cXTLSUwv1sCUt9Uw01Ew1NRRQPKiLTj0gIGIFp_2b2oLQFEYYkHSQB1OQAk0Bynm21OizQ1K0UoIGvDpQE8oxrNQE_3dE5ArrayIdLi1E1C7mutable7alignedEd5ArrayIdLi1E1C7mutable7alignedE__errcode__;
.visible .global .align 4 .u32 _ZN6cudapy8__main__26multiply_by_scalar_and_addB2v5B96cw51cXTLSUwv1sCUt9Uw01Ew1NRRQPKiLTj0gIGIFp_2b2oLQFEYYkHSQB1OQAk0Bynm21OizQ1K0UoIGvDpQE8oxrNQE_3dE5ArrayIdLi1E1C7mutable7alignedEd5ArrayIdLi1E1C7mutable7alignedE__tidx__;
.visible .global .align 4 .u32 _ZN6cudapy8__main__26multiply_

In [ ]:
##Note the compiler is not soo modern to perform ahead of time compilation

In [35]:
@cuda.jit
def multiply_by_scalar_fp32(arr, scalar, result):
    idx = cuda.grid(1)

    if idx < arr.size:
        result[idx] = arr[idx] * scalar

In [36]:
def test_multiply_scalar_fp32():
    arr = np.random.random(10000000).astype(np.float32)
    scalar = np.float32(8.8)
    result = np.zeros_like(arr, dtype=np.float32)

    tpb = 1024
    d_result = cuda.to_device(result)
    multiply_by_scalar_fp32[return_blocks_and_threads(tpb, arr.size), tpb](
        cuda.to_device(arr),scalar, d_result)

    print(d_result.copy_to_host())

In [37]:
test_multiply_scalar_fp32()

[8.201456  1.6162573 3.2755187 ... 1.8885994 2.3919206 8.245604 ]


In [38]:
for sig, assembly in multiply_by_scalar_fp32.inspect_asm().items():
    print(f"Signature - {sig}")
    print(f"Assembly - {assembly}")

Signature - (Array(float32, 1, 'C', False, aligned=True), float32, Array(float32, 1, 'C', False, aligned=True))
Assembly - //
// Generated by NVIDIA NVVM Compiler
//
// Compiler Build ID: CL-34097967
// Cuda compilation tools, release 12.4, V12.4.131
// Based on NVVM 7.0.1
//

.version 8.4
.target sm_61
.address_size 64

	// .globl	_ZN6cudapy8__main__23multiply_by_scalar_fp32B2v7B96cw51cXTLSUwv1sCUt9Uw01Ew1NRRQPKiLTj0gIGIFp_2b2oLQFEYYkHSQB1OQAk0Bynm21OizQ1K0UoIGvDpQE8oxrNQE_3dE5ArrayIfLi1E1C7mutable7alignedEf5ArrayIfLi1E1C7mutable7alignedE
.visible .global .align 4 .u32 _ZN6cudapy8__main__23multiply_by_scalar_fp32B2v7B96cw51cXTLSUwv1sCUt9Uw01Ew1NRRQPKiLTj0gIGIFp_2b2oLQFEYYkHSQB1OQAk0Bynm21OizQ1K0UoIGvDpQE8oxrNQE_3dE5ArrayIfLi1E1C7mutable7alignedEf5ArrayIfLi1E1C7mutable7alignedE__errcode__;
.visible .global .align 4 .u32 _ZN6cudapy8__main__23multiply_by_scalar_fp32B2v7B96cw51cXTLSUwv1sCUt9Uw01Ew1NRRQPKiLTj0gIGIFp_2b2oLQFEYYkHSQB1OQAk0Bynm21OizQ1K0UoIGvDpQE8oxrNQE_3dE5ArrayIfLi1E1C7mutab